In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [C:\Users\AndrewZ\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


### Visit the NASA Mars News Site

In [3]:
# In the next cell of your Jupyter notebook, we'll assign the url and instruct the browser to visit it.
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [ ]:
# In the next empty cell, we'll set up the HTML parser:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [ ]:
#pretty print of the site tree 
print (news_soup.prettify())
# list the tree : 
#list(news_soup.children)

In [ ]:
# We'll want to assign the title and summary text to variables we'll reference later. In the next empty cell, let's begin our scraping. Type the following:
slide_elem.find('div', class_='content_title').get_text()

In [ ]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
# Alternative method to run find: news_p = slide_elem.select_one('div.article_teaser_body')
news_p

### JPL Space Images Featured Image

In [ ]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [ ]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [ ]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [ ]:
# Find the relative image url :What we've done here is tell BeautifulSoup to look inside the <img /> tag for an image with a class of fancybox-image. Basically we're saying, "This is where the image we want lives—use the link that's inside these tags."

img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

In [ ]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

In [ ]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.head()

In [ ]:
#Ask David:  it's not the same table i see in the web site. I am missing, No earth Data in the table. 

# Scrape Mars Data: Mars Facts, Scape Tables
# Instead of scraping each row, or the data in each <td />, we're going to scrape the entire table with Pandas' .read_html() function.
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

In [ ]:
# Our data is live—if the table is updated, then we want that change to appear in Robin's app also
#Pandas also has a way to easily convert our DataFrame back into HTML-ready code using the .to_html() function
df.to_html()

In [ ]:
#D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

In [ ]:
###Hemispheres

In [9]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [10]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
for i in range(4):
    hemisphere = {}
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css('a.product-item img')[i].click()
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css('h2.title').text
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    # Finally, we navigate backwards
    browser.back()

In [11]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [ ]:
#Now that we've gathered everything on Robin's list, we can end the automated browsing session
browser.quit()